Join files of  Commercial Timber list

In [26]:
import pandas as pd
import textacy
import textacy.preprocess
import spacy
nlp = spacy.load('en') 
import re

file_loc = 'base_files_manual'
main_file = file_loc + '/'+  'Commercial_Timber.xlsx'
main_df = pd.read_excel(main_file,header=2,encoding="ISO-8859-1")

main_df = main_df[[
    'Coniferous vs. Nonconiferous', 
    'Type', 
    'Family', 
    'Scientific name ',
    'Total number of sources'
]]

main_df.reset_index(inplace=True)
main_df['genus'] = None
main_df['species'] = None
main_df['sub_species'] = None
main_df['coniferous'] = 0
main_df = main_df.rename(columns= {"Scientific name ":"sc_name"})




In [27]:
entity_id = 1
# Assumption that only 3 max variations of scientific names exist
columns = [
    'entity_id',
    'is_coniferous',
    'sc_name',
    'family'
]
new_df = pd.DataFrame(columns=columns)

var_kw = 'var.'
var_kws = ['misspelt', 'also as','as','mispelt']
sub_sp_kw = 'subsp.'

#parse out the parenthesis
def aux_1(phrase ,genus = '' , sp = '' ) :
    i1 = phrase.index('(')
    i2 = phrase.index(')')
    substr = phrase[i1:i2]
    substr = substr.strip('(')
    substr = substr.strip(')')
    substr = substr.strip()
    
    tmp = []
    doc = textacy.Doc(substr,lang='en')
   
    for t in doc :
        exclude_pos = ['VERB', 'CCONJ', 'ADV', 'ADP', 'PUNCT', 'SYM' , 'INTJ' , 'PART']
        if t.pos_ not in exclude_pos  and str(t) not in var_kws:
            tmp.append(str(t)) 
    
    if len(tmp) == 0:
        return ''
    
    if tmp[0][0].isupper():
        gn2 = tmp[0]
        if  len(tmp) == 1 :
            sp2 = sp
        else :
            sp2 = tmp[1]
        
    elif len(tmp) == 1 and tmp[0][0].islower() :
        gn2 = genus
        sp2 = tmp[0]
    tmp = ' '.join([gn2,sp2])
    return tmp
    


In [28]:
for i,row in  main_df.iterrows():
    
    sc_name = row["sc_name"]
    sc_name =  sc_name.replace('\xa0', " ")
    sc_name = sc_name.strip()
    sc_name = textacy.preprocess.normalize_whitespace(sc_name)
    
    family = row ['Family']
    _type = row['Type']
    sc_names = []
    
    if type(sc_name) == str :
        parts = sc_name.split(' ')
        
        # Case 1 : simple
        if len(parts) == 2 :
            # single sc_name , no subspecies
            num = 1
            sc_names = [' '.join([parts[0],parts[1]])]
        
        # Case 2 : if subspecies and parenthesis both present
        elif sub_sp_kw in sc_name and '(' in sc_name :
            
            parts_0 = sc_name.split(sub_sp_kw)
            parts_1 = parts_0[0].split(' ')
            
            g = parts_1[0].strip()
            sp = parts_1[1].strip()
            
            # remove the  () part an get the sub species
            i1 = parts_0[1].index('(')
            ssp = parts_0[1][0:i1]
            ssp = ssp.strip()
            
            n1 = ' '.join([g,sp,ssp])
            sc_names.append(n1)
            
            alt_name = aux_1(parts_0[1])
            sc_names.append(alt_name)
        
        elif sub_sp_kw in sc_name :
            n1 = sc_name.replace(sub_sp_kw,'')
            n1 = textacy.preprocess.normalize_whitespace(n1)
            sc_names.append(n1)
            
            parts = sc_name.split(sub_sp_kw)
            n2 =textacy.preprocess.normalize_whitespace(parts[0])
            sc_names.append(n2)
            
        elif var_kw in sc_name:   
            # 2 sc_name , no subspecies
            parts = sc_name.split(var_kw)
            parts_1 = parts[0].split(' ')
            parts_2 = parts[1]
            genus = parts_1[0].strip()
            sp1 =  parts_1[1].strip()
            sp2  = parts_2.strip() 
            n1 = ' '.join([genus,sp1])
            n2 = ' '.join([genus,sp2])
            sc_names.append(n1)
            sc_names.append(n2)
            
        elif '(' in sc_name :
            
            i1 = sc_name.index('(')
            part_1 = sc_name[0:i1]
            part_1.strip('(')
            part_1 = textacy.preprocess.normalize_whitespace(part_1)
            part_2 = part_1.split(' ')
            gn = part_2[0]
            sp = part_2[1]
            n1 = ' '.join([gn,sp])
            sc_names.append(n1)
            
            alt_name = aux_1(sc_name,gn,sp)
            sc_names.append(alt_name)
            
            
        elif ' x ' in sc_name :
            sc_name = sc_name.replace('x','')
            sc_name =  textacy.preprocess.normalize_whitespace(sc_name)
            parts = sc_name.split(' ')
            genus = parts[0].strip()
            sp  = parts[1].strip()
            n1 = ' '.join([genus,sp])
            sc_names.append(n1)
           
    
    # Coniferous or Non-Coniferous
    c_type = row['Coniferous vs. Nonconiferous']
    
    if c_type == 'Coniferous':
        con = 1
    else :
        con = 0
    tmp_dict = {}

    for scn in sc_names:
        if scn == None or scn == '' :
            continue
        tmp_dict['entity_id'] = entity_id
        tmp_dict['family'] = family
        tmp_dict['is_coniferous'] = con
        tmp_dict['sc_name'] = scn
        entity_id += 1
        new_df = new_df.append(tmp_dict,ignore_index=True)
    
new_df.to_csv('file1_processed.csv')

In [29]:
columns = list(new_df.columns)
print (columns)
print(max(list(new_df['entity_id'])))

['entity_id', 'is_coniferous', 'sc_name', 'family']
1753


In [30]:
# read in file 2 
file2 = file_loc + '/'+  'file_2.xlsx'
df2 = pd.read_excel(file2,header=0,encoding="ISO-8859-1")
print(list(df2.columns))

df = df2[['Family', 'Scientific name ']]
df = df.rename(columns = {'Scientific name ' : 'sc_name', 'Family' : 'family' })
cur_entity_id = entity_id
print('Current entity id', cur_entity_id)

for i, row in df.iterrows():
    sc_name = row['sc_name']
    family = row['family']
    # search in older dataframe
    r = main_df.index[(main_df['sc_name'] == sc_name)]
    if len(r) > 0 :
        print(r)
    else :
        tmp_dict = {
            'entity_id' : cur_entity_id,
            'sc_name' : sc_name,
            'family' : family,
            'is_coniferous' : None 
        }
        cur_entity_id += 1
        
        new_df = new_df.append(tmp_dict, ignore_index=True)

new_df.to_csv('CommercialTimber_all_processed.csv')    

['Family', 'Scientific name ', 'Total number of sources', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']
Current entity id 1754
